In [1]:
import json, re, spacy, string
import pandas as pd
from nltk.corpus import stopwords

2024-04-29 14:29:27.528695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1. Read in data

### V1:

In [2]:
f = open("./data/raw/DebateNet-migr15.json", mode="r", encoding="utf-8")
df_v1 = json.load(f)
df_v1 = pd.json_normalize(df_v1)
df_v1.to_csv(sep="\t", encoding="utf-8", index=False)
df_v1

,deleted,project_id,user_id,id,doc_id,ts,data.actorvalues,data.quote,data.end,data.text,data.actorclusters,data.date_uncertain,data.cdate,data.begin,data.claimvalues,data.cpos,data.$$hashKey,data.id_origin_str,data.provenance,data.xid
0,False,1,0,3981,111670,1544796845950,[per_Flüchtlingsrat-Geschäftsführerin\n Naujoks],"""Wir brauchen diese Mindeststandards",3404,,[{'pers': ['Flüchtlingsrat-Geschäftsführerin ...,0.0,20150113,3368,"[300, 309]",1,NaN,NaN,NaN,NaN
1,False,1,0,4435,135202,1544796406629,[org_Grünen],ein kollektives Bleiberecht für die in Hamburg...,1361,,"[{'pers': [], 'outside': True, 'parties': ['Gr...",0.0,20150408,1280,"[200, 211]",1,NaN,NaN,NaN,NaN
2,False,1,0,3979,111670,1544105536145,[org_Stadt Schwerte],Die Stadt Schwerte will Asylsuchende in einem ...,503,,"[{'pers': [], 'outside': False, 'parties': [],...",0.0,20150113,389,"[200, 204]",1,NaN,NaN,NaN,NaN
3,False,1,0,4436,135202,1544796406673,[org_Hamburgs rot-grüne Koalition],"eine von den Grünen bislang abgelehnte ""gesch...",1614,,"[{'pers': [], 'outside': True, 'parties': ['Gr...",0.0,20150408,1482,"[400, 408]",1,NaN,NaN,NaN,NaN
4,False,1,0,3975,121911,1550062876849,[per_Gerd Bosbach],Ich begrüße die Zuwanderung nach Deutschland.,4598,,"[{'pers': ['Gerd Bosbach'], 'outside': True, '...",0.0,20151022,4553,"[100, 101]",-1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,False,1,0,5788,111107,1549285457888,"[per_Sigmar Gabriel, per_Frank-Walter Steinmei...",Die hat nun ausdrücklich die SPD ins Spiel geb...,2309,,"[{'pers': ['Sigmar Gabriel'], 'outside': False...",0.0,20151009,2124,"[100, 102]",1,NaN,NaN,NaN,NaN
1811,False,1,0,wfjAR2wBGMXeBDVZQgCn,134094,1564571878053,"[20, per_Malu Dreyer, per_Torsten Albig, org_SPD]",Die Regierungschefs von Rheinland-Pfalz und Sc...,2891,,"[{'pers': ['Malu Dreyer'], 'outside': False, '...",0.0,20150913,2729,"[800, 808]",1,NaN,ggcHR2sBwZjXaQ5Fiqte,NaN,NaN
1812,False,1,0,4094,122931,1544513390964,"[org_Wirtschaftsflügel seiner Partei, org_CDU,...",Unterstützung erhielt er jetzt aber vom Wirtsc...,1473,,"[{'pers': ['Wolfgang Steiger'], 'outside': Fal...",0.0,20150202,1401,"[100, 108]",1,NaN,NaN,NaN,NaN
1813,False,1,0,4351,135492,1573053971236,[org_Bundesministerium],Neben Einmal-Zuschüssen will das Ministerium d...,3905,,"[{'pers': [], 'outside': False, 'parties': [],...",0.0,20150618,3727,"[800, 803]",1,NaN,NaN,NaN,59.0


### V2:

In [3]:
df_v2 = pd.read_json(path_or_buf="./data/raw/DebateNet-migr15v2.jsonl", lines=True)
df_v2.to_csv(sep="\t", encoding="utf-8", index=False)
df_v2

,quote,cdate,name,parties,end,begin,doc_id,claimvalues,anno_id,cpos
0,Das ist widersinnig und schlichtweg nicht nach...,2015-12-21,Daniel Bax,[],2003,1945,108168,"[200, 207, 500, 504]",2uQ6-m4BGPKI_RhCPQOu,-1
1,CDU/CSU Horst Seehofer droht der Kanzlerin mit...,2015-10-28,Horst Seehofer,[CSU],136,30,143569,"[100, 102]",puThhG8BGPKI_RhCixJn,1
2,Nun hat Seehofer der Bundesregierung am Dienst...,2015-10-28,Horst Seehofer,[CSU],884,733,143569,"[100, 102]",ouThhG8BGPKI_RhCixI0,1
3,Vor zwei Wochen drohte Bayerns Ministerpräsid...,2015-10-14,Horst Seehofer,[CSU],521,317,143569,"[800, 801]",p-ThhG8BGPKI_RhCixJ0,1
4,"Im Bayerischen Landtag sagte Seehofer, er wüns...",2015-10-28,Horst Seehofer,[CSU],3382,3241,143569,"[100, 102]",o-ThhG8BGPKI_RhCixJC,1
...,...,...,...,...,...,...,...,...,...,...
3676,Die Regierungschefs von Rheinland-Pfalz und Sc...,2015-09-13,Torsten Albig,[SPD],2891,2729,134094,"[800, 808]",wfjAR2wBGMXeBDVZQgCn,1
3677,Unterstützung erhielt er jetzt aber vom Wirtsc...,2015-02-02,Wolfgang Steiger,[CDU],1473,1401,122931,"[100, 108]",4094,1
3678,Unterstützung erhielt er jetzt aber vom Wirtsc...,2015-02-02,CDU_Wirtschaftsflügel,[CDU],1473,1401,122931,"[100, 108]",4094,1
3679,Neben Einmal-Zuschüssen will das Ministerium d...,2015-06-18,Bundesministerium,[],3905,3727,135492,"[800, 803, 805]",4351,1


# 2. Merge V1 and V2:

In [4]:
df_v1_cleaned = pd.DataFrame({"source": "1", "date": df_v1["data.cdate"], "quote": df_v1["data.quote"], "values": df_v1["data.claimvalues"]}) 
df_v2_cleaned = pd.DataFrame({"source": "2", "date": [i.replace("-", "") for i in df_v2["cdate"]], "quote": df_v2["quote"], "values": df_v2["claimvalues"]}) 

df_all = pd.concat([df_v1_cleaned, df_v2_cleaned], ignore_index=True)
df_all['values'] = df_all['values'].apply(lambda x: str(x)) # Converst the lists in "values" to string. Otherwise drop_duplicates() would yield error
df_all = df_all.drop_duplicates(subset=['quote', 'values'], keep="last", ignore_index=True)

df_all

,source,date,quote,values
0,1,20151215,"Amnesty International bezweifelt, dass eine s...","['800', '801']"
1,1,20150126,"Gefordert werden ein Burka-Verbot, die strikte...","['100', '102', '200', '207', '800', '812']"
2,1,20150225,"Wenn der Staat das nicht will, soll er seine A...","['400', '402']"
3,1,20150122,"Zweitens heißt es, dem Rassismus und der Demok...","['700', '706']"
4,1,20151007,Laut Gesetz müssen sie aber zunächst in einer ...,"['200', '201']"
...,...,...,...,...
3291,2,20151009,Die hat nun ausdrücklich die SPD ins Spiel geb...,"['100', '102']"
3292,2,20150913,Die Regierungschefs von Rheinland-Pfalz und Sc...,"['800', '808']"
3293,2,20150202,Unterstützung erhielt er jetzt aber vom Wirtsc...,"['100', '108']"
3294,2,20150618,Neben Einmal-Zuschüssen will das Ministerium d...,"['800', '803', '805']"


# 3. Simplify the label format

In [5]:
# Minimize labels
label_minimal = []
for i in df_all["values"]:
    i = re.sub("[\[\]']", "", i)

    currentLabelSet = set()
    for label in i.split(", "):
        currentLabelSet.add(label[0])
    label_minimal.append(currentLabelSet)

df_all.insert(loc=4, column="values_minimal", value=label_minimal)
df_all

,source,date,quote,values,values_minimal
0,1,20151215,"Amnesty International bezweifelt, dass eine s...","['800', '801']",{8}
1,1,20150126,"Gefordert werden ein Burka-Verbot, die strikte...","['100', '102', '200', '207', '800', '812']","{8, 2, 1}"
2,1,20150225,"Wenn der Staat das nicht will, soll er seine A...","['400', '402']",{4}
3,1,20150122,"Zweitens heißt es, dem Rassismus und der Demok...","['700', '706']",{7}
4,1,20151007,Laut Gesetz müssen sie aber zunächst in einer ...,"['200', '201']",{2}
...,...,...,...,...,...
3291,2,20151009,Die hat nun ausdrücklich die SPD ins Spiel geb...,"['100', '102']",{1}
3292,2,20150913,Die Regierungschefs von Rheinland-Pfalz und Sc...,"['800', '808']",{8}
3293,2,20150202,Unterstützung erhielt er jetzt aber vom Wirtsc...,"['100', '108']",{1}
3294,2,20150618,Neben Einmal-Zuschüssen will das Ministerium d...,"['800', '803', '805']",{8}


# 4. Preprocessing

In [6]:
quote = df_all["quote"]
stopwords_list = stopwords.words("german")

In [7]:
nlp = spacy.load("de_core_news_sm")

In [8]:
quote_lemmatized = []
for text in quote:
    current_lemmatized_text = ""
    doc = nlp(text)
    
    for s in doc.sents:
        for w in s:
            if w.lemma_.lower() not in stopwords_list \
            and w.lemma_ != "--" \
            and w.text not in string.punctuation \
            and re.fullmatch("\d+[.,]?(\d+)?", w.lemma_) is None:
                current_lemmatized_text += w.lemma_ + " "
   
    quote_lemmatized.append(current_lemmatized_text)

In [9]:
df_all.insert(loc=3, column="quote_lemmatized", value=quote_lemmatized)
df_all

,source,date,quote,quote_lemmatized,values,values_minimal
0,1,20151215,"Amnesty International bezweifelt, dass eine s...",Amnesty International bezweifeln Vorgeleg ru...,"['800', '801']",{8}
1,1,20150126,"Gefordert werden ein Burka-Verbot, die strikte...",fordern Burka-Verbot strikt Begrenzung Einwand...,"['100', '102', '200', '207', '800', '812']","{8, 2, 1}"
2,1,20150225,"Wenn der Staat das nicht will, soll er seine A...",Staat sollen Arbeit gut Mensch Schutz gewähren,"['400', '402']",{4}
3,1,20150122,"Zweitens heißt es, dem Rassismus und der Demok...",zweitens heißen Rassismus Demokratieverachtung...,"['700', '706']",{7}
4,1,20151007,Laut Gesetz müssen sie aber zunächst in einer ...,laut Gesetz müssen zunächst Erstaufnahmeeinric...,"['200', '201']",{2}
...,...,...,...,...,...,...
3291,2,20151009,Die hat nun ausdrücklich die SPD ins Spiel geb...,ausdrücklich SPD Spiel bringen Vizekanzler Sig...,"['100', '102']",{1}
3292,2,20150913,Die Regierungschefs von Rheinland-Pfalz und Sc...,Regierungschef Rheinland-Pfalz Schleswig-Holst...,"['800', '808']",{8}
3293,2,20150202,Unterstützung erhielt er jetzt aber vom Wirtsc...,Unterstützung erhalten Wirtschaftsflügel Partei,"['100', '108']",{1}
3294,2,20150618,Neben Einmal-Zuschüssen will das Ministerium d...,neben Einmal-Zuschüss Ministerium Kommune dadu...,"['800', '803', '805']",{8}


In [10]:
df_all.to_csv("./data/preprocessed/DebateNet-migr15_all_lemmatized.tsv", sep="\t", index=False, encoding="utf-8")